In [3]:
import pandas as pd

# Define las rutas a tus archivos de resultados
ruta_core = '../results/comparacion_metricas_core.csv'
ruta_paper = '../results/comparacion_metricas_paper.csv'
# Carga los CSV en DataFrames
df_core = pd.read_csv(ruta_core)
df_paper = pd.read_csv(ruta_paper)

df_paper.style \
    .format({
        "POL": "{:.3f}", "BA-POL": "{:.3f}", "BAL": "{:.2%}",
        "MAC": "{:.3f}", "MAO": "{:.3f}", "DENS": "{:.3f}",
        "ISO": "{:.3f}", "CC+": "{:.3f}", "CC-": "{:.3f}"
    }) \
    .background_gradient(
        cmap='RdYlGn',  # Rojo (bajo) a Verde (alto)
        subset=["POL", "BA-POL", "BAL", "ISO", "CC+", "CC-"]
    ) \
    .set_caption("🚀 Tabla 2: Métricas Detalladas")

,Dataset,Algoritmo,POL,BA-POL,BAL,SIZE,ISO,CC+,CC-,CC_intra_S1,CC_intra_S2
0,Plebiscito PRE,Eigensign (Determinista),5.742,4.698,81.82%,120,0.176,0.635,0.880,0.291981,0.581879
1,Plebiscito PRE,Random Eigensign (Probabilístico),2.373,1.727,72.77%,660,0.329,0.603,0.800,0.112295,0.357895
2,Plebiscito PRE,Local Search (beta=0.01),4.631,3.643,78.67%,536,0.452,0.594,0.803,0.248669,0.404152
3,Plebiscito PRE,Local Search (beta=0.005),3.850,3.702,96.15%,816,0.527,0.620,0.798,0.370160,0.208573
4,Plebiscito PRE,SCG (max_obj),5.988,4.459,74.47%,82,0.140,0.649,0.890,0.601742,0.378076
5,Plebiscito PRE,SCG (randomized),2.240,1.655,73.88%,699,0.339,0.603,0.780,0.332854,0.081166
6,Plebiscito PRE,SCG (bansal),2.100,0.900,42.86%,10,0.013,1.000,0.667,0.795238,1.000000
7,Plebiscito POST,Eigensign (Determinista),5.316,1.418,26.67%,19,0.060,0.881,0.730,0.833333,0.844211
8,Plebiscito POST,Random Eigensign (Probabilístico),1.756,1.741,99.15%,468,0.336,0.574,0.676,0.084100,0.279588
9,Plebiscito POST,Local Search (beta=0.01),3.547,3.547,100.00%,400,0.469,0.556,0.697,0.273097,0.437945


In [4]:
import pandas as pd

# Función para aplicar color a las celdas de variación (sin cambios)
def color_celda_variacion(val):
    """
    Devuelve un estilo con color de fondo sólido y color de fuente negro.
    """
    color_fondo = ''
    if val > 0:
        color_fondo = 'lightgreen'
    elif val < 0:
        color_fondo = '#F8B6A8'
    
    if not color_fondo:
        return ''
        
    return f'background-color: {color_fondo}; color: black;'


# --- Carga de Datos ---
try:
    ruta_paper = '../results/comparacion_metricas_paper.csv'
    # --- MODIFICADO: Cargar el nuevo archivo de resultados TF-IDF ---
    ruta_tfidf = '../results/comparacion_terminos_tfidf.csv'
    
    df_paper = pd.read_csv(ruta_paper)
    df_tfidf = pd.read_csv(ruta_tfidf)
except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo {e.filename}.")
    print("Por favor, verifica que la ruta sea correcta y que el archivo exista.")
    exit()

# --- Tablas de Métricas Pre y Post Evento (sin cambios) ---
df_pre = df_paper[df_paper['Dataset'].str.contains('PRE', case=False)].set_index('Algoritmo')
df_post = df_paper[df_paper['Dataset'].str.contains('POST', case=False)].set_index('Algoritmo')

metric_cols = ["POL", "BA-POL", "BAL", "SIZE", "ISO", "CC+", "CC-", "CC_intra_S1", "CC_intra_S2"]
metric_cols = [col for col in metric_cols if col in df_pre.columns and col in df_post.columns]

# Tabla 1: PRE-EVENTO
tabla_pre_styled = df_pre[metric_cols].style \
    .format("{:.3f}", na_rep="-") \
    .set_caption("📊 Tabla 1: Métricas Pre-Evento")
print("--- Mostrando Tabla Pre-Evento (sin colores) ---")
display(tabla_pre_styled)

# Tabla 2: POST-EVENTO
df_post_con_cambio = df_post[metric_cols].copy()
for col in metric_cols:
    cambio = (df_post[col] - df_pre[col]) / df_pre[col].abs()
    df_post_con_cambio[f'{col}_Δ%'] = cambio
delta_cols = [col for col in df_post_con_cambio.columns if col.endswith('_Δ%')]
tabla_post_styled = df_post_con_cambio.style \
    .format("{:.3f}", subset=metric_cols, na_rep="-") \
    .format("{:+.1%}", subset=delta_cols, na_rep="-") \
    .set_caption("🚀 Tabla 2: Métricas Post-Evento y Variación vs. Pre-Evento") \
    .apply(lambda x: x.map(color_celda_variacion), subset=delta_cols)
print("\n--- Mostrando Tabla Post-Evento con Variación y Estilo de Color Sólido ---")
display(tabla_post_styled)


# --- MODIFICADO: Tabla para Top 5 Términos Distintivos (TF-IDF) ---

# Prepara los datos: combina término y score TF-IDF para mejor visualización
# El score se formatea a 3 decimales
df_tfidf['Termino (Score)'] = df_tfidf['Termino'] + ' (' + df_tfidf['Score_TFIDF'].map('{:.3f}'.format) + ')'

# El script de análisis ya devuelve los términos ordenados, así que head(5) toma los 5 mejores
df_top5 = df_tfidf.groupby(['Dataset', 'Algoritmo', 'Cluster']).head(5).reset_index()

# Crea un ranking (1 a 5) para poder pivotar la tabla
df_top5['Rank'] = df_top5.groupby(['Dataset', 'Algoritmo', 'Cluster']).cumcount() + 1

# Pivota la tabla para tener un formato ancho y limpio
df_terminos_pivot = df_top5.pivot_table(
    index=['Dataset', 'Algoritmo'],
    columns=['Cluster', 'Rank'],
    values='Termino (Score)', # Usa la nueva columna combinada
    aggfunc='first'
).fillna('')

# Renombra las columnas para mayor claridad
df_terminos_pivot.columns = [f'{cluster}_Top{rank}' for cluster, rank in df_terminos_pivot.columns]

# Aplica un estilo limpio a la tabla de términos
tabla_terminos_styled = df_terminos_pivot.style \
    .set_caption("💬 Tabla 3: Top 5 Términos Distintivos por Cluster (TF-IDF)") \
    .set_properties(**{'text-align': 'left', 'border': '1px solid #ddd'}) \
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}])

print("\n--- Mostrando Tabla de Top 5 Términos TF-IDF ---")
display(tabla_terminos_styled)

--- Mostrando Tabla Pre-Evento (sin colores) ---


,POL,BA-POL,BAL,SIZE,ISO,CC+,CC-,CC_intra_S1,CC_intra_S2
Algoritmo,,,,,,,,,
Eigensign (Determinista),5.742,4.698,0.818,120.000,0.176,0.635,0.880,0.292,0.582
Random Eigensign (Probabilístico),2.373,1.727,0.728,660.000,0.329,0.603,0.800,0.112,0.358
Local Search (beta=0.01),4.631,3.643,0.787,536.000,0.452,0.594,0.803,0.249,0.404
Local Search (beta=0.005),3.850,3.702,0.962,816.000,0.527,0.620,0.798,0.370,0.209
SCG (max_obj),5.988,4.459,0.745,82.000,0.140,0.649,0.890,0.602,0.378
SCG (randomized),2.240,1.655,0.739,699.000,0.339,0.603,0.780,0.333,0.081
SCG (bansal),2.100,0.900,0.429,10.000,0.013,1.000,0.667,0.795,1.000



--- Mostrando Tabla Post-Evento con Variación y Estilo de Color Sólido ---


,POL,BA-POL,BAL,SIZE,ISO,CC+,CC-,CC_intra_S1,CC_intra_S2,POL_Δ%,BA-POL_Δ%,BAL_Δ%,SIZE_Δ%,ISO_Δ%,CC+_Δ%,CC-_Δ%,CC_intra_S1_Δ%,CC_intra_S2_Δ%
Algoritmo,,,,,,,,,,,,,,,,,,
Eigensign (Determinista),5.316,1.418,0.267,19.000,0.060,0.881,0.730,0.833,0.844,-7.4%,-69.8%,-67.4%,-84.2%,-65.7%,+38.8%,-17.1%,+185.4%,+45.1%
Random Eigensign (Probabilístico),1.756,1.741,0.991,468.000,0.336,0.574,0.676,0.084,0.280,-26.0%,+0.9%,+36.2%,-29.1%,+2.2%,-4.8%,-15.5%,-25.1%,-21.9%
Local Search (beta=0.01),3.547,3.547,1.000,400.000,0.469,0.556,0.697,0.273,0.438,-23.4%,-2.6%,+27.1%,-25.4%,+3.9%,-6.4%,-13.2%,+9.8%,+8.4%
Local Search (beta=0.005),2.480,2.337,0.942,639.000,0.551,0.504,0.621,0.386,0.199,-35.6%,-36.9%,-2.0%,-21.7%,+4.5%,-18.6%,-22.2%,+4.3%,-4.8%
SCG (max_obj),5.476,1.711,0.312,21.000,0.070,0.835,0.680,0.844,0.533,-8.5%,-61.6%,-58.0%,-74.4%,-50.1%,+28.7%,-23.6%,+40.3%,+41.1%
SCG (randomized),1.702,1.688,0.992,506.000,0.356,0.522,0.666,0.293,0.080,-24.0%,+2.0%,+34.3%,-27.6%,+5.0%,-13.5%,-14.7%,-11.8%,-2.0%
SCG (bansal),3.000,0.833,0.278,23.000,0.053,0.688,0.758,0.600,0.670,+42.9%,-7.4%,-35.2%,+130.0%,+299.8%,-31.2%,+13.6%,-24.6%,-33.0%



--- Mostrando Tabla de Top 5 Términos TF-IDF ---
